In [1]:
import os
import requests
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def sendNotification(app_name, message, additional_message='', event_name='program_log'):
    my_key = 'dtFZmutEg9ANUtSEpAU7Tu'
    data = {
        "value1" : app_name,
        "value2" : message,
        "value3" : additional_message
    }
    url = "https://maker.ifttt.com/trigger/" + event_name + "/with/key/" + my_key
    
    r = requests.post(url, data=data)
    
    return r

In [3]:
def waitForElement(driver, by_what=By.XPATH, element_info='', delay=20, do_quit=True):
    try:
        elem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((by_what, element_info)))
        print("Page is ready!")
        return elem
    except TimeoutException:
        print("Loading took too much time!")
        print("quiting while waiting for element:", element_info)
        if do_quit:
            report = "quiting while waiting for element: " + element_info
            send_report_and_close(report, driver)
        return None

In [4]:
# sendNotification('Hello', 'Hello', event_name='program_log')

In [5]:
def send_report_and_close(report, driver):
    for i in range(10):
        r = sendNotification('Jupyter Health Report', report)
        if r.status_code == 200:
            break
        else:
            print("Hasn't send retrying... " + str(i) + " of 10")
  
    driver.close()
    exit()

In [6]:
from config import *

url = 'https://webvpn.xmu.edu.cn/login'
driver = None


path_to_driver = os.path.join(os.path.curdir, 'chromedriver')
driver = webdriver.Chrome(path_to_driver)


driver.get(url)

if '<html dir="ltr" lang="en"><head>' in driver.page_source:
    report = 'URL is incorrect.'
    send_report_and_close(report, driver )

if webvpn[0] == '' or webvpn[1] == '':
    report = 'webVPN login and/or password is empty'
    send_report_and_close(report, driver )

In [7]:
login_field = driver.find_element_by_xpath("//input[@id='user_name']")
login_field.click()
print('clicking login')
# login_field.clear()
login_field.send_keys(webvpn[0])

password_field = driver.find_element_by_xpath("//input[@type='password']")
password_field.click()
print('clicking password')
# password_field.clear()
password_field.send_keys(webvpn[1])

loging_button = waitForElement(driver, element_info="//button[@id='login']")
loging_button.click()

xmuxgbutton = waitForElement(driver, element_info="//p[contains(text(),'https://xmuxg.xmu.edu.cn/')]")
xmuxgbutton.click()

clicking login
clicking password
Page is ready!
Page is ready!


In [8]:
# sleep(5)
driver.switch_to.window(driver.window_handles[0])
driver.close()
driver.switch_to.window(driver.window_handles[-1])

loaded_url = driver.current_url

In [9]:
if url != loaded_url:
    print("logged into webvpn successfully")


logged into webvpn successfully


In [10]:
# '''
#             logging into webvpn
# '''
# login_field = waitForElement(driver, "//input[@id='user_name']")
# login_field.click()
# print('clicking login')
# # login_field.clear()
# login_field.send_keys(webvpn[0])

# password_field = driver.find_element_by_xpath("//input[@type='password']")
# password_field.click()
# print('clicking password')
# # password_field.clear()
# password_field.send_keys(webvpn[1])

# loging_button = waitForElement(driver, element_info="//button[@id='login']")
# loging_button.click()

# xmuxgbutton = waitForElement(driver, element_info="//p[contains(text(),'https://xmuxg.xmu.edu.cn/')]")
# xmuxgbutton.click()

# # sleep(5)
# driver.switch_to.window(driver.window_handles[0])
# driver.close()
# driver.switch_to.window(driver.window_handles[-1])

# loaded_url = driver.current_url

In [11]:
report = ''
# url = 'https://xmuxg.xmu.edu.cn/app/214'
# path_to_driver = os.path.join(os.path.curdir, 'chromedriver')
#
# driver = webdriver.Chrome(path_to_driver)
# driver.get(url)


In [12]:
loaded_url = driver.current_url
if 'login' in loaded_url:
    # log in here then
    print('waiting to click')
    loging_button = waitForElement(driver, element_info="//button[contains(.,'统一身份认证')]")

    loging_button.click()


waiting to click
Page is ready!


In [13]:
    login_field = driver.find_element_by_xpath("//input[@id='username']")
    login_field.click()
    print('clicking login')
    # login_field.clear()
    login_field.send_keys(login[1])

    password_field = driver.find_element_by_xpath("//input[@id='password']")
    password_field.click()
    print('clicking password')
    # password_field.clear()
    password_field.send_keys(password[1])

clicking login
clicking password


In [14]:
    # press login/登录
    loging_button = driver.find_element_by_xpath("//button[contains(@class, 'auth_login_btn')]")
    loging_button.click()
    
    if loaded_url != driver.current_url:
        print("logged in successfully")
        report += 'Login OK.'
    else:
        print("Hasn't logged in")
        report = "Hasn't logged in. Check log:pass"
        send_report_and_close(report, driver)
        # TODO: send_report_and_close()

#
# /html/body/div[1]/div[@id='main']/div[@class='page-container container-fluid']/div[@class='page-content row']/div[@class='col-sm-12 page left-show']/div[@id='mainPage-page']/div[@class='main-p']/div[@class='shadow_box box_wrap_2']/div[2]/div[@class='box_main box_flex']/div[@class='app_child box_flex'][1]/div[@class='grow_1 box_flex column justify_center']/div[@class='title box_flex']

logged in successfully


In [15]:
print("closing nav bar")
nav_bar = waitForElement(driver, element_info="//div[@class='menu-toggle pull-left']//i[@class='maticon']")
nav_bar.click()

print("selecting daily health section")
dhr_section = waitForElement(driver, element_info="//div[contains(text(),'Daily Health Report')]")

closing nav bar
Page is ready!
selecting daily health section
Page is ready!


In [16]:
dhr_section.click()

### New window choosing

In [17]:
sleep(3)
driver.switch_to.window(driver.window_handles[0])
driver.close()

driver.switch_to.window(driver.window_handles[-1])

In [18]:
# 健康打卡
menu_button = waitForElement(driver, element_info="//div[contains(@class, 'tab')][2]")

Page is ready!


In [19]:
# CHECK DATE
from datetime import datetime

curdate = datetime.today().strftime('%Y-%m-%d')

print(curdate)

2021-04-15


In [20]:
if len(driver.find_elements_by_xpath("//span[text()[contains(.,'" + curdate + "')]]")) > 0:
    print("date is correct:", curdate)
    report += "Date OK."
else:
    print("there's something wrong with date "+ curdate +", but we'll continue anyway")
    report += "Date BAD."

date is correct: 2021-04-15


In [21]:
# CHECK TIME
hours = int(datetime.today().strftime('%H'))
minutes = int(datetime.today().strftime('%M'))

if (hours >= 19 and minutes >= 30) or hours < 7:
    print("too late for the daily health report")
    report += "Time BAD."
    send_report_and_close(report, driver)
    # TODO: send_report_and_close()
else:
    print("time's alright:", hours, ':', minutes)
    report += "Time OK."

time's alright: 10 : 21


In [22]:
# Finally getting to the reporting part,
# menu_button = driver.find_element_by_xpath("//div[contains(@class, 'tab')][2]")
# menu_button = driver.find_element_by_xpath("//div[text()='我的菜单')]")
menu_button.click()


In [23]:
# waiting for tab to load up properly
_ = waitForElement(driver,
                           element_info="//span[contains(text(),'Can you hereby declare that all the information')]")

# for el in driver.find_elements_by_xpath("//div[contains(@class, 'v-select btn-block info-value btn-group')]"):
#     print(el.id, "-", el.tag_name, "-", el.text)


# confirmation = driver.find_element_by_css_selector('div.page-container.application-page div.app-detail-page-form div.middle.middle-top div.preview-container:nth-child(1) div.preview-wrap div.preview-page.pc-view div.container-fluid.form-preview-content.pc-view.form-view:nth-child(4) div.form-style div.row.cell-div-pc:nth-child(31) div.form-group.form-cell.col-sm-12 div.v-select.btn-block.info-value.btn-group div.form-control.dropdown-toggle span.dropdown-icon > i.maticon')
confirmation_field = driver.find_elements_by_xpath("//div[contains(@class, 'v-select btn-block info-value btn-group')]")[-1]


Page is ready!


In [ ]:
if len(driver.find_elements_by_xpath("//span[text()[contains(.,'是 Yes')]]")) > 0:
    print("confirmation is already yes, consider checking website yourself")
    report += 'Yes already.'
            # clicking outside
            # driver.find_element_by_xpath("//body").click()
else:
    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    # hoping that the last element is the confirmation one
    confirmation_field.click()
    print("2) chose confirmation field")
    yes_button = driver.find_element_by_xpath(
                "//span[text()[contains(.,'是 Yes')]]/ancestor::label[@class='btn-block']")

    yes_button.click()

In [ ]:
save_button = driver.find_element_by_xpath("//span[contains(@class, 'form-save position-absolute')]")
save_button.click()

In [ ]:
'''
    POPUP ALERT
'''
driver.switch_to.alert.accept()
print("5) clicked OK on an alert window")

'''
    保存成功
'''
saved_suc = waitForElement(driver, element_info="//pre[contains(@class, 'message')]", do_quit=False)

report += 'Saved OK. Login: ' + login[0]


In [ ]:
# sendNotification("DH", report)
# driver.close()
print("It should be done by now")
send_report_and_close(report, driver)